<a href="https://colab.research.google.com/github/kristinkx/KristinKX/blob/main/youtube_trending_videos_analysis_(1).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
from google.colab import files
uploaded = files.upload()

TypeError: ignored

In [ ]:
!pip install pandasql

In [ ]:
youtube_df = pd.read_csv('Trending videos on youtube dataset.csv')
youtube_df.head(3)

In [ ]:
youtube_df.describe()

In [ ]:
youtube_df.info()

In [ ]:
# unique values in each column
youtube_df.nunique()

In [ ]:
print(youtube_df.shape)

In [ ]:
# check null values
null_count = youtube_df.isnull().sum()
null_count

In [ ]:
# use for loop to check null 
# result is consistent with the info() function above

for col in youtube_df.columns:
    pct_missing = np.mean(youtube_df[col].isnull())
    print('{} - {}%'.format(col, pct_missing))

In [ ]:
# convert durationSec to min and sec and add the two columns to df
from datetime import datetime 

youtube_df['play_min'] = youtube_df['durationSec']//60
# youtube_df['play_min']

youtube_df['play_sec'] = youtube_df['durationSec'] - youtube_df['play_min']*60
# youtube_df['play_sec'] 
youtube_df.head()

In [ ]:
youtube_df.videoDescription.fillna('No description available', inplace = True)
youtube_df.head(3)

In [ ]:
# find the top 10 most liked youtube videos in the dataset (like likecount)
# use panda sql
import pandasql as ps # SQL on Pandas Dataframe

top_100_liked_query = '''

SELECT videoTitle, videoDescription,channelId, 
channelTitle, videoId, durationSec, likeCount,
commentCount, videoCategoryLabel,play_min, play_sec

FROM youtube_df
ORDER BY likeCount DESC
LIMIT 100

'''
top_100_liked_df = ps.sqldf(top_100_liked_query, globals())
top_100_liked_df.head()

In [ ]:
# tokenize the text from the youtube df
import nltk
from nltk.tokenize import word_tokenize
import string

In [ ]:
# # all the stopwords are in lowercase format
from nltk.corpus import stopwords
import nltk
nltk.download('stopwords')

stop_words = set(stopwords.words("english"))

In [ ]:
# make a list of video category label
top_cat = top_100_liked_df['videoCategoryLabel'].values.tolist()
# top_des 

In [ ]:
from nltk.tokenize import word_tokenize
import string

def tokenized_cat(videos):
  answer = []
  
  for single in videos:
    for others in nltk.word_tokenize(single): 
      others = others.lower()
      if others not in stop_words and others.isalpha():
        answer.append(others)

  return answer


In [ ]:
# call the function and flatten the tokens
import nltk
nltk.download('punkt')

top_cat_list = list(tokenized_cat(top_cat))
# top_cat_tokens = [*set(top_cat_tokens)]
# top_cat_list 

In [ ]:
from collections import Counter
# find the 20 most commen words in the category label
top_cat_most_common = Counter(top_cat_list).most_common(15)
top_cat_most_common

In [ ]:
# randomly sample 1 million tokens from the the top description tokens
import random
top_cat_sample = Counter(random.sample(top_cat, 100))

In [ ]:
from nltk.corpus.reader import wordlist
from nltk.translate.meteor_score import wordnetsyn_match
import wordcloud
import numpy as np
from PIL import Image
import os

from matplotlib import pyplot as plt

import matplotlib.pyplot as plt 
def plot_word_cloud(list):

  things_for_cloud = ""
  words = []
    
  for element in list:
    words.append(element[0])
    things_for_cloud += ' '.join(words) + ''
  
  # mask =np.array(Image.open('/kaggle/input/star-image/star.png'))
  # generate word cloud
  actual_cloud = wordcloud.WordCloud(width = 500, height = 500, prefer_horizontal= 0.88, \
                                     mask = None, contour_width = 0, contour_color = 'blue', \
                                     scale = 1, min_font_size = 6, font_step = 1, max_words = 100,\
                                     background_color = 'black', max_font_size = None, mode = 'RGB',\
                                     relative_scaling='auto', color_func = None, collocations = True,\
                                     colormap = 'rainbow', normalize_plurals= True, repeat = False, \
                                     random_state = 1, include_numbers = False).generate(things_for_cloud)
  # plot word cloud
  plt.figure(figsize = (8, 8), facecolor = None)
  plt.imshow(actual_cloud)
  plt.axis('off')
  plt.tight_layout(pad = 0)

  plt.show()

plot_word_cloud(top_cat_most_common)

In [ ]:
# plotting histograms
youtube_df.hist(bins = 60, figsize =(20, 15)); 

In [ ]:
# find correlation between viewcount and likecount
youtube_df['viewCount'].corr(youtube_df['likeCount'])

In [ ]:
import matplotlib
import matplotlib.pyplot as plt

# show correlation between the youtube videos view count and like count in the df
# very strong positive correlation between the two variables

plt.scatter(x = youtube_df['viewCount'], y = youtube_df['likeCount']) 
plt.title ('YouTube view count vs. like count')
plt.xlabel('YouTube view count')
plt.ylabel('like count')
plt.show()

In [ ]:
import seaborn as sns
# use seaborn to show the strong positive correlation
sns.regplot(x = 'likeCount', y = 'viewCount', data = youtube_df, scatter_kws={"color": 'blue'}, line_kws = {'color': "red"});

In [ ]:
# find correlation between durationSec and viewcount
# almost no correlation between the two variables

youtube_df['durationSec'].corr(youtube_df['viewCount'])

In [ ]:
# find out the correlations

youtube_df.corr(method = 'pearson')

In [ ]:
# heatmap
# use heatmap to demonstration correlation; brighter colors mean higher corr
fig, ax = plt.subplots(figsize = (16, 8))
correlation_matrix = youtube_df.corr(method = 'pearson')
sns.heatmap(correlation_matrix, annot = True, fmt = '1.2f', annot_kws = {'size' : 10}, linewidth = 1)
plt.show()

In [ ]:
# map the correlation between commentCount and all other variables

correlation_new = abs(youtube_df.corr())
plt.figure(figsize=(10,8))
plt.plot(correlation_new["commentCount"].sort_values(ascending=False)[1:],label="Correlation",color="red")
plt.ylabel("Correlation")
plt.xlabel("YouTube videos indicators")
plt.legend()
plt.tight_layout()
plt.grid(True)
plt.show()